In [1]:
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv


In [2]:
load_dotenv()

True

In [3]:
class LLMstate(TypedDict):
    topic:str
    outline:str
    essay:str


In [4]:
model=ChatOpenAI()

In [5]:
graph=StateGraph(LLMstate)

In [6]:
def outline_node(state:LLMstate)->LLMstate:
    topic=state['topic']
    prompt=f'Give me the outline for this topic = {topic} which I will use to create an essay in future'
    outline=model.invoke(prompt).content
    state['outline']=outline
    return state


In [7]:
def essay_node(state:LLMstate)->LLMstate:
    topic=state['topic']
    outline=state['outline']
    prompt=f'Write an essay of 200 words on this topic = {topic} with the given outline below {outline}'
    essay=model.invoke(prompt).content
    state['essay']=essay
    return state


In [8]:
graph.add_node('outline_node',outline_node)
graph.add_node('essay_node',essay_node)

graph.add_edge(START,'outline_node')
graph.add_edge('outline_node','essay_node')
graph.add_edge('essay_node', END)

workflow=graph.compile()




In [14]:
initial_state={'topic':'India'}
final_state=workflow.invoke(initial_state)
print('*'*30)
print(final_state['topic'])
print('*'*30)
print(final_state['outline'])
print('*'*30)
print(final_state['essay'])

******************************
India
******************************
I. Introduction
    A. Brief overview of India's geography, history, and culture
    B. Thesis statement: India is a diverse and vibrant country with a rich history and culture that has shaped its identity and global influence.

II. Geography of India
    A. Overview of India's geographical features and climate
    B. Description of India's major regions and landscapes

III. History of India
    A. Early civilizations and ancient empires in India
    B. The impact of colonialism on India's history
    C. India's struggle for independence and the formation of modern India

IV. Culture of India
    A. Overview of India's diverse culture, languages, and religions
    B. Traditional arts and music in India
    C. Festivals and holidays celebrated in India

V. Global influence of India
    A. India's growing economy and influence on the world stage
    B. India's contributions to fields such as science, technology, and lite